In [1]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import tensorflow as tf
import tensorflow_addons as tfa
from tensorflow.keras import *
import numpy as np
import copy
from functools import partial
import sys, os
os.chdir('..')
join = os.path.join(os.getcwd(), '_global')
sys.path.extend([join])
from _global.config import *
from _global.funcs import *
sys.path.extend([RESEARCH])
from object_detection.utils import config_util
from object_detection.protos import pipeline_pb2
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder
from google.protobuf import text_format
from IPython import get_ipython
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.INFO)
ipython = get_ipython()

In [2]:
def filename_wo_extension(name):
    filename = name.split("/")
    filename = filename[len(filename)-1]
    filename = filename.split(".")
    return filename[0]

def download_and_install_model():
    if not os.path.exists(MODELS_PATH):
        os.mkdir(MODELS_PATH)
    
    if not os.path.exists(PRETRAINED_MODELS_PATH):
        os.mkdir(PRETRAINED_MODELS_PATH)

    model_wo_version = os.path.join(MODELS_PATH, PRETRAINED_MODEL_NAME)
    
    if not os.path.exists(model_wo_version):
        os.mkdir(model_wo_version)
        extractTar(PRETRAINED_MODEL_DOWNLOAD, PRETRAINED_MODELS_PATH, PRETRAINED_MODEL_NAME)

    if not os.path.exists(MODEL):
        os.mkdir(MODEL)

    os.chdir(MODEL)

    exec(ipython.transform_cell('!copy {} {}'.format(os.path.join(PRETRAINED_MODELS_PATH, PRETRAINED_MODEL_NAME, "pipeline.config"), MODEL)))
    
    os.chdir(INSTALLATION_PATH)

    return MODEL

In [3]:
PIPELINE_CONFIG_PATH = os.path.join(download_and_install_model(), "pipeline.config")

        1 Datei(en) kopiert.


In [4]:
config = config_util.get_configs_from_pipeline_file(PIPELINE_CONFIG_PATH)

In [5]:
pipeline_config = pipeline_pb2.TrainEvalPipelineConfig()

In [6]:
pipeline_config = pipeline_pb2.TrainEvalPipelineConfig()
with tf.io.gfile.GFile(PIPELINE_CONFIG_PATH, "r") as f:                                                                                                                                                                                                                     
    proto_str = f.read()                                                                                                                                                                                                                                          
    text_format.Merge(proto_str, pipeline_config)  

In [7]:
pipeline_config.model.ssd.num_classes = 1
pipeline_config.train_config.batch_size = 8
pipeline_config.train_config.fine_tune_checkpoint = os.path.join(PRETRAINED_MODELS_PATH, PRETRAINED_MODEL_NAME, 'checkpoint', 'ckpt-0')
pipeline_config.train_config.fine_tune_checkpoint_type = "detection"
pipeline_config.train_input_reader.label_map_path= LABEL_MAP_PATH
pipeline_config.train_input_reader.tf_record_input_reader.input_path[:] = [OUTPUT_TRAIN_TFRECORD]
pipeline_config.eval_input_reader[0].label_map_path = LABEL_MAP_PATH
pipeline_config.eval_input_reader[0].tf_record_input_reader.input_path[:] = [OUTPUT_VALIDATION_TFRECORD]

In [8]:
config_text = text_format.MessageToString(pipeline_config)                                                                                                                                                                                                        
with tf.io.gfile.GFile(PIPELINE_CONFIG_PATH, "wb") as f:                                                                                                                                                                                                                     
    f.write(config_text)  

In [9]:
cmd_str = "python {} --model_dir={} --pipeline_config_path={} --num_train_steps=2000"
cmd_str = "python {} --logtostderr --train_dir={} --pipeline_config_path={} --num_train_steps=10000"
train_command = cmd_str.format(TRAINING_SCRIPT, MODEL, PIPELINE_CONFIG_PATH)
train_command

'python C:\\Users\\niklas\\Desktop\\Face_Recognition_Ullsperger\\LogicAIFaceRecProject\\workspace\\model_main_tf2.py --logtostderr --train_dir=C:\\Users\\niklas\\Desktop\\Face_Recognition_Ullsperger\\LogicAIFaceRecProject\\workspace\\models\\ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8\\v5 --pipeline_config_path=C:\\Users\\niklas\\Desktop\\Face_Recognition_Ullsperger\\LogicAIFaceRecProject\\workspace\\models\\ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8\\v5\\pipeline.config --num_train_steps=10000'

In [ ]:
!{train_command}

In [ ]:
 
eval_cmd_str = "python {} --pipeline_config_path={} --model_dir={} --checkpoint_dir={}"
eval_cmd = eval_cmd_str.format(TRAINING_SCRIPT, PIPELINE_CONFIG_PATH, MODEL, 
os.path.join(MODEL, 'ckpt-3'))
eval_cmd

In [ ]:
!{eval_cmd}

In [ ]:
for example in tf.compat.v1.python_io.tf_record_iterator(OUTPUT_TRAIN_TFRECORD):
    print(tf.train.Example.FromString(example))